In [48]:
import numpy as np
import pandas as pd
from common import *

In [49]:
segments = 15
time_step = 15
w = 32
pred_steps = 5
start_time = '06:00:00'
end_time = '19:00:00'

In [50]:
initial = pd.read_csv("data/bus_running_times_feature_added_all.csv")

In [51]:
busses_new = initial[['trip_id','deviceid','direction','segment','date','start_time','end_time','run_time','run_time_in_seconds']]

In [52]:
busses_new.dropna(inplace=True)


C:\Users\chath\AppData\Local\Temp\ipykernel_17920\1412130111.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  busses_new.dropna(inplace=True)


In [53]:
mask = busses_new['segment'] <= segments

In [54]:
busses_new = busses_new[mask]

In [55]:
# Convert the 'date' column to a datetime object
busses_new['date'] = pd.to_datetime(busses_new['date'])

# Create a boolean mask to filter rows with dates on or after 10/1/2022
mask = busses_new['date'] >= '2022-10-01'

# Create the train and test dataframes
busses_train = busses_new[~mask]
busses_test = busses_new[mask]

In [56]:
busses_train.head()

,trip_id,deviceid,direction,segment,date,start_time,end_time,run_time,run_time_in_seconds
0,1.0,262.0,1.0,1.0,2021-10-01,06:39:49,06:40:58,0 days 00:01:09,69.0
1,1.0,262.0,1.0,2.0,2021-10-01,06:42:12,06:45:42,0 days 00:03:30,210.0
2,1.0,262.0,1.0,3.0,2021-10-01,06:45:42,06:53:58,0 days 00:08:16,496.0
3,1.0,262.0,1.0,4.0,2021-10-01,06:54:04,06:57:19,0 days 00:03:15,195.0
4,1.0,262.0,1.0,5.0,2021-10-01,06:57:19,06:58:56,0 days 00:01:37,97.0


In [57]:
df_avg_run_time = getAvgRuntimeWithSegment(busses_train,time_step)

C:\Users\chath\Documents\academics sem 5\CS3501 Data Science and Engineering Project\code\common.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataframe['start_time'] = pd.to_datetime(dataframe['start_time'])


In [58]:
df = handleOutliers(df_avg_run_time)

In [59]:
df_normalized = normalizeSTD(df, segments, start_time, end_time, time_step,df)

In [60]:
df_normalized.head()

,date,start_time,segment,normalized_avg_run_time
0,2021-10-01,06:00:00,1,0.0
1,2021-10-01,06:00:00,2,0.0
2,2021-10-01,06:00:00,3,0.0
3,2021-10-01,06:00:00,4,0.0
4,2021-10-01,06:00:00,5,0.0


In [61]:
input_data_x, input_data_y = prepareModelInput(df_normalized, w, segments, pred_steps)

In [62]:
x_train, x_test, y_train, y_test = splitData(input_data_x, input_data_y, 0.2)

In [63]:
model = build_model(w,pred_steps,segments)

In [65]:
import tensorflow as tf

# Set GPU memory growth to avoid memory allocation issues
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


In [66]:
trainedModel = trainModel(model, x_train, y_train, 64, 3, 0.1)

Epoch 1/3
  21/2106 [..............................] - ETA: 1:14:01 - loss: 0.3698

KeyboardInterrupt: 